##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Document search with embeddings

<table class="tfo-notebook-buttons" align="left">
      <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/tutorials/document_search"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemini-api/tutorials/document_search.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/gemini-api/tutorials/document_search.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview

This example demonstrates how to use the Gemini API to create embeddings so that you can perform document search. You will use the Python client library to build a word embedding that allows you to compare search strings, or questions, to document contents.

In this tutorial, you'll use embeddings to perform document search over a set of documents to ask questions related to the Google Car.

## Prerequisites

You can run this quickstart in Google Colab.

To complete this quickstart on your own development environment, ensure that your environment meets the following requirements:

-  Python 3.9+
-  An installation of `jupyter` to run the notebook.

## Setup

First, download and install the Gemini API Python library.

In [2]:
!pip install -U -q google-generativeai

In [3]:
import textwrap
import numpy as np
import pandas as pd

import google.generativeai as genai

# Used to securely store your API key
from google.colab import userdata

from IPython.display import Markdown

### Grab an API Key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GOOGLE_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`

In [5]:
# Or use `os.getenv('API_KEY')` to fetch an environment variable.
API_KEY=userdata.get('API_KEY')

genai.configure(api_key=API_KEY)

Key Point: Next, you will choose a model. Any embedding model will work for this tutorial, but for real applications it's important to choose a specific model and stick with it. The outputs of different models are not compatible with each other.

**Note**: At this time, the Gemini API is [only available in certain regions](https://ai.google.dev/gemini-api/docs/available-regions).

In [6]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


## Embedding generation

In this section, you will see how to generate embeddings for a piece of text using the embeddings from the Gemini API.

### API changes to Embeddings with model embedding-001

For the new embeddings model, embedding-001, there is a new task type parameter and the optional title (only valid with task_type=`RETRIEVAL_DOCUMENT`).

These new parameters apply only to the newest embeddings models.The task types are:

Task Type | Description
---       | ---
RETRIEVAL_QUERY	| Specifies the given text is a query in a search/retrieval setting.
RETRIEVAL_DOCUMENT | Specifies the given text is a document in a search/retrieval setting.
SEMANTIC_SIMILARITY	| Specifies the given text will be used for Semantic Textual Similarity (STS).
CLASSIFICATION	| Specifies that the embeddings will be used for classification.
CLUSTERING	| Specifies that the embeddings will be used for clustering.

Note: Specifying a `title` for `RETRIEVAL_DOCUMENT` provides better quality embeddings for retrieval.

In [7]:
title = "The next generation of AI for developers and Google Workspace"
sample_text = ("Title: The next generation of AI for developers and Google Workspace"
    "\n"
    "Full article:\n"
    "\n"
    "Gemini API & Google AI Studio: An approachable way to explore and prototype with generative AI applications")

model = 'models/text-embedding-004'
embedding = genai.embed_content(model=model,
                                content=sample_text,
                                task_type="retrieval_document",
                                title=title)

print(embedding)

{'embedding': [-0.0021609084, -0.0031644478, -0.060120754, -0.0071218493, 0.0008775481, 0.040581927, 0.04457149, 0.03552469, -0.04746538, 0.008888608, -0.02795826, 0.011335696, -0.0024438712, 0.0030851837, -0.018796137, -0.055550937, 0.03142646, 0.0006549187, -0.11370059, 0.063708074, -0.021750016, -0.021367034, -0.09982074, -0.008604756, -0.03330058, -0.012815642, 0.07153146, 0.037064772, 0.022970116, 0.043331202, 0.010670608, 0.040685344, 0.036361404, -0.036222056, -0.01779936, -0.014820963, 0.0053205034, -0.017382707, 0.07044941, 0.002021254, -0.018208738, 0.01755808, 0.006493214, 0.12724239, -0.023805201, 0.010057808, -0.0006948924, 0.070856266, -0.05645728, 0.018311134, 0.090462275, 0.021575555, -0.06656089, 0.026865067, -0.0034812444, -0.0011228676, -0.06535635, -0.0018169138, 0.08672994, 0.028747609, -0.024817271, 0.004653876, -0.058998518, 0.032061696, -0.022604035, -0.015454275, -0.013758674, 0.021129983, -0.047893394, 0.02573244, 0.01302824, -0.018002003, -0.039879415, 0.0234

## Building an embeddings database

Here are three sample texts to use to build the embeddings database. You will use the Gemini API to create embeddings of each of the documents. Turn them into a dataframe for better visualization.

In [59]:
DOCUMENT1 = {
    "title": "Pantaloncini corti blu",
    "content": "#estate #pantaloncini #blu #bermuda"}
DOCUMENT2 = {
    "title": "Maglietta bianca",
    "content": "#maglietta #tshirt #bianca #estiva"}
DOCUMENT3 = {
    "title": "Giacca in piuma d'oca",
    "content": "#inverno #giacca #piuma"}
DOCUMENT4 = {
    "title": "Giacca in pelle",
    "content": "#primavera #autunno #giacca #pelle"}
DOCUMENT5 = {
    "title": "Pantaloni termici sintetici",
    "content": "#inverno #pantaloni #termici"}
DOCUMENT6 = {
    "title": "Jeans",
    "content": "#jeans #pantaloni"}
DOCUMENT7 = {
    "title": "Cappello areato",
    "content": "#estate #cappello #traspirante #areato"}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3, DOCUMENT4, DOCUMENT5, DOCUMENT6, DOCUMENT7]

Organize the contents of the dictionary into a dataframe for better visualization.

In [60]:
df = pd.DataFrame(documents)
df.columns = ['Title', 'Text']
df

,Title,Text
0,Pantaloncini corti blu,#estate #pantaloncini #blu #bermuda
1,Maglietta bianca,#maglietta #tshirt #bianca #estiva
2,Giacca in piuma d'oca,#inverno #giacca #piuma
3,Giacca in pelle,#primavera #autunno #giacca #pelle
4,Pantaloni termici sintetici,#inverno #pantaloni #termici
5,Jeans,#jeans #pantaloni
6,Cappello areato,#estate #cappello #traspirante #areato


Get the embeddings for each of these bodies of text. Add this information to the dataframe.

In [61]:
# Get the embeddings of each text and add to an embeddings column in the dataframe
def embed_fn(title, text):
  return genai.embed_content(model=model,
                             content=text,
                             task_type="retrieval_document",
                             title=title)["embedding"]

df['Embeddings'] = df.apply(lambda row: embed_fn(row['Title'], row['Text']), axis=1)
df

,Title,Text,Embeddings
0,Pantaloncini corti blu,#estate #pantaloncini #blu #bermuda,"[-0.0019876547, -0.014734267, 0.04808125, -0.0..."
1,Maglietta bianca,#maglietta #tshirt #bianca #estiva,"[-0.029447883, -0.02066756, 0.025725853, -0.01..."
2,Giacca in piuma d'oca,#inverno #giacca #piuma,"[-0.03561864, 0.03512419, 0.011823384, -0.0241..."
3,Giacca in pelle,#primavera #autunno #giacca #pelle,"[-0.028802548, 0.005428984, 0.020488774, -0.04..."
4,Pantaloni termici sintetici,#inverno #pantaloni #termici,"[-0.038922433, 0.009753581, 0.0008522512, -0.0..."
5,Jeans,#jeans #pantaloni,"[-0.034064304, -0.018504601, 0.03743465, -0.03..."
6,Cappello areato,#estate #cappello #traspirante #areato,"[0.022777807, 0.0033081074, 7.724639e-05, -0.0..."


## Document search with Q&A

Now that the embeddings are generated, let's create a Q&A system to search these documents. You will ask a question about hyperparameter tuning, create an embedding of the question, and compare it against the collection of embeddings in the dataframe.

The embedding of the question will be a vector (list of float values), which will be compared against the vector of the documents using the dot product. This vector returned from the API is already normalized. The dot product represents the similarity in direction between two vectors.

The values of the dot product can range between -1 and 1, inclusive. If the dot product between two vectors is 1, then the vectors are in the same direction. If the dot product value is 0, then these vectors are orthogonal, or unrelated, to each other. Lastly, if the dot product is -1, then the vectors point in the opposite direction and are not similar to each other.

Note, with the new embeddings model (`embedding-001`), specify the task type as `QUERY` for user query and `DOCUMENT` when embedding a document text.

Task Type | Description
---       | ---
RETRIEVAL_QUERY	| Specifies the given text is a query in a search/retrieval setting.
RETRIEVAL_DOCUMENT | Specifies the given text is a document in a search/retrieval setting.

In [62]:
query = "Scarponi invernali: #scarponi #neri #pesanti #riscaldanti"
model = 'models/text-embedding-004'

request = genai.embed_content(model=model,
                              content=query,
                              task_type="retrieval_query")

Use the `find_best_passage` function to calculate the dot products, and then sort the dataframe from the largest to smallest dot product value to retrieve the relevant passage out of the database.

In [63]:
def find_best_passage(query, dataframe):
  """
  Compute the distances between the query and each document in the dataframe
  using the dot product.
  """
  query_embedding = genai.embed_content(model=model,
                                        content=query,
                                        task_type="retrieval_query")
  dot_products = np.dot(np.stack(dataframe['Embeddings']), query_embedding["embedding"])
  idx = np.argmax(dot_products)
  return [dataframe.iloc[idx]['Title'], dataframe.iloc[idx-1]['Title'], dataframe.iloc[idx-2]['Title']] # Return text from index with max value

View the most relevant document from the database:

In [64]:
passage = find_best_passage(query, df)
passage

['Pantaloni termici sintetici', 'Giacca in pelle', "Giacca in piuma d'oca"]

## Question and Answering Application

Let's try to use the text generation API to create a Q & A system. Input your own custom data below to create a simple question and answering example. You will still use the dot product as a metric of similarity.

In [ ]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = textwrap.dedent("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
  strike a friendly and converstional tone. \
  If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

In [ ]:
prompt = make_prompt(query, passage)
print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below.   Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.   However, you are talking to a non-technical audience, so be sure to break down complicated concepts and   strike a friendly and converstional tone.   If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: 'How do you shift gears in the Google car?'
  PASSAGE: 'Shifting Gears  Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position i

Choose one of the Gemini content generation models in order to find the answer to your query.

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.5-pro
models/gemini-1.5-flash


In [ ]:
model = genai.GenerativeModel('gemini-1.5-pro-latest')
answer = model.generate_content(prompt)

In [ ]:
Markdown(answer.text)

The provided passage does not contain information about how to shift gears in a Google car, so I cannot answer your question from this source.

## Next steps

To learn how to use other services in the Gemini API, see the [Python quickstart](https://ai.google.dev/tutorials/python_quickstart).

To learn more about how you can use embeddings, see these  other tutorials:

 * [Anomaly Detection with Embeddings](https://ai.google.dev/gemini-api/tutorials/anomaly_detection)
 * [Clustering with Embeddings](https://ai.google.dev/gemini-api/tutorials/clustering_with_embeddings)
 * [Training a Text Classifier with Embeddings](https://ai.google.dev/gemini-api/tutorials/text_classifier_embeddings)